# Imports

In [1]:
#!pip install statsmodels matplotlib

In [2]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, DoubleType
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import numpy as np
import numpy as np
from pyspark.sql.functions import col
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.types import DoubleType
import time
import scipy.stats as stats

# Initialize Spark session

In [3]:
spark = SparkSession.builder.master("spark://spark-master:7077").appName("ManualQRDecomposition").getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/26 17:25:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# 1) Funktionen für Lineare Regression mit QR Decomposition

## 1.1) Lösen eines LGS durch Rückwärts Einsetzen

In [4]:
def backward_substitution(R, b):
    """
    Solves the upper triangular system Ax = y for x using backward substitution.
    
    Args:
    R: Upper triangular matrix
    y (array): The right-hand side vector.
    
    Returns:
    array: Solution vector b.
    """
    n = len(b)
    x = np.zeros(n)
    
    for i in range(n-1, -1, -1):
        sum_val = 0
        for j in range(i+1, n):
            sum_val += R[i, j] * x[j]
        
        if abs(R[i, i]) > 1e-10:  # Check for numerical stability
            x[i] = (b[i] - sum_val) / R[i, i]
        else:
            x[i] = 0
            
    return x

## 1.2) QR Zerlegung

### 1.2.1) Gram Schmidt ohne Sparkoptimierung

In [5]:
def manual_qr_decomposition(X_array):
    """
    Manual implementation of QR decomposition using Gram-Schmidt process.
    
    Args:
        X_array: Input matrix as numpy array
        
    Returns:
        Q: Orthogonal matrix
        R: Upper triangular matrix
    """
    n, m = X_array.shape
    Q = np.zeros((n, m))
    R = np.zeros((m, m))
    
    for j in range(m):
        v = X_array[:, j].copy()
        
        for i in range(j):
            R[i, j] = 0
            for k in range(n):
                R[i, j] += Q[k, i] * X_array[k, j]
            
            for k in range(n):
                v[k] = v[k] - R[i, j] * Q[k, i]
        
        norm = 0
        for k in range(n):
            norm += v[k] * v[k]
        norm = np.sqrt(norm)
        
        R[j, j] = norm
        
        if norm > 1e-10:  # Avoid division by very small numbers
            for k in range(n):
                Q[k, j] = v[k] / norm
        else:
            for k in range(n):
                Q[k, j] = 0
    
    return Q, R

### 1.2.1) Gram Schmidt mit Sparkoptimierung

In [6]:
def gram_schmidt(X_df):
    X_rdd = X_df.rdd.cache()
    m = len(X_df.first().features)
    Q = []
    R = np.zeros((m, m))

    for j in range(m):
        v = X_rdd.map(lambda row: row.features[j]).collect()
        
        for i in range(j):
            R[i, j] = np.dot(Q[i], v)
            v -= R[i, j] * Q[i]

        R[j, j] = np.linalg.norm(v)
        Q.append(v / R[j, j])
    
    R_dict = {i: R[i, :] for i in range(m)}

    Q_df = spark.createDataFrame([Row(features=DenseVector(q)) for q in zip(*Q)])

    return Q_df, R_dict

## 1.3) Datensatz simulieren

### 1.3.1) Datensatz simulieren ohne Sparkoptimierung

In [7]:
def create_data_numpy(n, p, beta_true):
    np.random.seed(42)
    X = np.random.rand(n, p)
    X = np.column_stack([np.ones(X.shape[0]), X])
    y = X @ beta_true + np.random.randn(n) * 0.1
    return X, y

### 1.3.2) Datensatz simulieren mit Sparkoptimierung

In [8]:
def create_data_spark(n_samples, n_features, beta_true, noise_std=0.1, partition_size=10000):
    """
    Generates synthetic data optimized for distributed processing.
    
    Args:
        spark: SparkSession
        n: Number of observations
        p: Number of features
        beta_true: True coefficients
        num_partitions: Number of partitions for distributed data
    """
    def generate_partition(partition_size):
        X = np.random.randn(partition_size, n_features)
        X = np.column_stack([np.ones(partition_size), X])
        y = X @ beta_true + np.random.normal(0, noise_std, partition_size)
        return [(Vectors.dense(x), float(y_i)) for x, y_i in zip(X, y)]
    
    num_partitions = max(n_samples // partition_size, spark.sparkContext.defaultParallelism)
    samples_per_partition = n_samples // num_partitions
    
    rdd = (spark.sparkContext
           .parallelize(range(num_partitions), num_partitions)
           .flatMap(lambda _: generate_partition(samples_per_partition)))
    
    return spark.createDataFrame(rdd, ["features", "y"])

## 1.4) Funktion zur Durchführung der linearen Regression 

- simuliert einen Datensatz
- führt die lineare Regression durch
- berechnet alle Metriken, die auch statmodels `summary()` ausgibt
- misst die Zeit

### 1.4.1 Lineare Regression mit Sparkoptimierung

In [9]:
def linear_regression_manual_qr(X, y):
    start_time = time.time()
    Q, R = manual_qr_decomposition(X)

    # Calculate Qt * y without collecting Q into memory
    Qt_y = np.array(Q.rdd.map(lambda row: sum(row.features[i] * y[i] for i in range(len(row.features)))).collect())
    beta = backward_substitution(R, Qt_y)

    # Calculate residuals and other statistics without collecting X or y
    residuals = y - X.rdd.map(lambda row: np.dot(row.features, beta)).collect()
    n = X.count()
    k = len(beta)  # Assuming that beta includes the intercept
    df_residuals = n - k

    # Compute sum of squared errors (SSE) and total sum of squares (SST) locally
    SSE = np.sum(np.square(residuals))
    mean_y = y.mean()  # Assuming y is already a local object
    SST = np.sum(np.square(y - mean_y))

    # R-squared and Adjusted R-squared calculations
    r_squared = 1 - (SSE / SST)
    adjusted_r_squared = 1 - ((1 - r_squared) * (n - 1) / df_residuals)

    # F-statistic calculation
    MSR = (SST - SSE) / (k - 1)
    f_statistic = MSR / (SSE / df_residuals)
    p_value_f = 1 - stats.f.cdf(f_statistic, k - 1, df_residuals)

    end_time = time.time()
    elapsed_time = end_time - start_time

    results = {
        'Observations': n,
        'R-squared': r_squared,
        'Adjusted R-squared': adjusted_r_squared,
        'F-statistic': f_statistic,
        'p-value (F-statistic)': p_value_f,
        'Residual standard error': np.sqrt(SSE / df_residuals),
        'Degrees of freedom': df_residuals
    }

    return results, elapsed_time


### 1.4.2 Optimiert für Spark

In [10]:
def fit_ols_manual(df):
    """
    Fit OLS using manual QR decomposition and backward substitution, with extended statistics.
    """
    start_time = time.time()
    
    # Step 1: Calculate combined QR decomposition across partitions
    def process_partition(iterator):
        # Local arrays for QR decomposition
        X_local = []
        y_local = []
        for row in iterator:
            X_local.append(row.features.toArray())
            y_local.append(row.y)
        
        # Early exit if partition is empty
        if not X_local:
            return []
        
        X_local = np.vstack(X_local)
        y_local = np.array(y_local)
        
        # Perform local QR decomposition
        Q_local, R_local = manual_qr_decomposition(X_local)
        
        # Calculate local Q^T y
        Qty_local = np.dot(Q_local.T, y_local)
        
        return [(R_local, Qty_local)]
    
    # Aggregate QR decomposition results
    results = df.rdd.mapPartitions(process_partition).reduce(lambda a, b: (
        a[0] + b[0],  # Sum R matrices
        a[1] + b[1]   # Sum Q^T y vectors
    ))
    
    R_total, Qty_total = results
    
    # Step 2: Solve for beta using the aggregated QR components
    beta = backward_substitution(R_total, Qty_total)
    
    # Step 3: Calculate residuals and additional statistics in a second RDD pass
    def compute_stats(iterator):
        # Local arrays for computing residuals
        X_local = []
        y_local = []
        for row in iterator:
            X_local.append(row.features.toArray())
            y_local.append(row.y)
        
        if not X_local:
            return []
        
        X_local = np.vstack(X_local)
        y_local = np.array(y_local)
        y_pred = np.dot(X_local, beta)
        residuals = y_local - y_pred
        
        # Compute statistics locally
        ss_res = np.sum(residuals ** 2)
        ss_tot = np.sum((y_local - np.mean(y_local)) ** 2)
        
        return [(ss_res, ss_tot, residuals)]
    
    # Aggregate residual statistics
    stats_results = df.rdd.mapPartitions(compute_stats).reduce(lambda a, b: (
        a[0] + b[0],  # Sum of squared residuals
        a[1] + b[1],  # Total sum of squares
        np.concatenate([a[2], b[2]])  # All residuals for skew and kurtosis
    ))
    
    ss_res, ss_tot, all_residuals = stats_results
    r_squared = 1 - (ss_res / ss_tot)
    n_samples = df.count()
    p = len(beta)
    
    # Calculate advanced statistics
    mse = ss_res / (n_samples - p)
    f_stat = ((ss_tot - ss_res) / p) / (ss_res / (n_samples - p - 1))
    skewness = stats.skew(all_residuals)
    kurtosis = stats.kurtosis(all_residuals, fisher=False)
    
    # More computations (AIC, BIC, etc.) can be added here using `n_samples`, `p`, `mse`, etc.
    
    computation_time = time.time() - start_time
    
    summary = {
        'r_squared': r_squared,
        'adj_r_squared': 1 - (1 - r_squared) * (n_samples - 1) / (n_samples - p - 1),
        'mse': mse,
        'f_statistic': f_stat,
        'skewness': skewness,
        'kurtosis': kurtosis,
        'computation_time': computation_time,
        'n_samples': n_samples,
        'n_features': p
    }
    
    return beta, summary

## 1.5) Funktion für Durchführung des Benchmarks

# Berechnung für Numpy

In [11]:
def run_benchmark_numpy(n_list, beta_true, p, repetitions=10):
    results = []

    for n in n_list:
        times = []
        for _ in range(repetitions):
            X, y = create_data_numpy(n, p, beta_true)
            beta, elapsed_time = linear_regression_manual_qr(X, y)
            times.append(elapsed_time)

        avg_time = np.mean(times)
        std_time = np.std(times)
        results.append([n, avg_time, std_time])

    return results

### Berechnung für eine Range

In [12]:

def run_benchmark_range(max_n, p, beta_true, step_size=100000, repetitions=10):
    """Run benchmarking with fixed step size increments."""
    #n_values = np.arange(step_size, max_n + step_size, step_size, dtype=int)
    n_values = np.logspace(5, np.log10(max_n), 10, dtype=int)
    results = []
    
    for n in n_values:
        times = []
        r_squares = []
        successful_runs = 0
        
        print(f"\nBenchmarking n={n:,} samples...")
        
        for i in range(repetitions):
            try:
                df = create_data_spark(n, p, beta_true)
                start_time = time.time()
                beta, summary = fit_ols_manual(df)
                elapsed_time = time.time() - start_time
                
                times.append(elapsed_time)
                r_squares.append(summary['r_squared'])
                successful_runs += 1
                
                print(f"  Run {i+1}: time={elapsed_time:.2f}s, R²={summary['r_squared']:.4f}")
                
            except Exception as e:
                print(f"  Run {i+1} failed: {str(e)}")
        
        if successful_runs > 0:
            result = {
                'n_samples': n,
                'n_features': p,
                'mean_time': np.mean(times),
                'std_time': np.std(times),
                'mean_r_squared': np.mean(r_squares),
                'std_r_squared': np.std(r_squares),
                'successful_runs': successful_runs
            }
            results.append(result)
            6
            print(f"\nSummary for n={n:,}:")
            print(f"  Mean time: {result['mean_time']:.2f}s ± {result['std_time']:.2f}s")
            print(f"  Mean R²: {result['mean_r_squared']:.4f} ± {result['std_r_squared']:.4f}")
    
    return pd.DataFrame(results)

### Berechnung für einen festen Wert

In [13]:

def run_benchmark_point(n, p, beta_true, repetitions=1):
    """Run benchmarking for a single sample size n."""
    results = []
    times = []
    r_squares = []
    successful_runs = 0
    
    print(f"\nBenchmarking n={n:,} samples...")
    
    for i in range(repetitions):
        try:
            df = create_data_spark(n, p, beta_true)
            start_time = time.time()
            beta, summary = fit_ols_manual(df)
            elapsed_time = time.time() - start_time
            
            times.append(elapsed_time)
            r_squares.append(summary['r_squared'])
            successful_runs += 1
            
            print(f"  Run {i+1}: time={elapsed_time:.2f}s, R²={summary['r_squared']:.4f}")
            
        except Exception as e:
            print(f"  Run {i+1} failed: {str(e)}")
    
    if successful_runs > 0:
        result = {
            'n_samples': n,
            'n_features': p,
            'mean_time': np.mean(times),
            'std_time': np.std(times),
            'mean_r_squared': np.mean(r_squares),
            'std_r_squared': np.std(r_squares),
            'successful_runs': successful_runs
        }
        results.append(result)
        
        print(f"\nSummary for n={n:,}:")
        print(f"  Mean time: {result['mean_time']:.2f}s ± {result['std_time']:.2f}s")
        print(f"  Mean R²: {result['mean_r_squared']:.4f} ± {result['std_r_squared']:.4f}")
    
    return pd.DataFrame(results)


# 2) Durchführung des Benchmarks und Ausgeben der Ergebnisse

## 2.1) Durchführung

In [14]:
#n_values = [200000, 500000, 1000000, 5000000, 10000000]
#n_features = 7
#beta_true = np.random.randn(n_features + 1)
#benchmark_results = run_benchmark_numpy(n_list=n_values, beta_true=beta_true, p=n_features) 

### Benchmark Point

In [ ]:
n_features = 7
beta_true = np.random.randn(n_features + 1)  # +1 for intercept
results_point = run_benchmark_point(n=10000000, p=n_features, beta_true=beta_true)

### Benchmark Range

In [ ]:
n_features = 7
beta_true = np.random.randn(n_features + 1)  # +1 for intercept
results_range = run_benchmark_range(max_n=10000000, p=n_features, beta_true=beta_true, step_size=100000)

## 2.2) Ausgeben der Ergebnisse

In [ ]:
for i in benchmark_results:
    print("\n Datenzeilen: ", i[0])
    print("Laufzeit: ", i[1])

In [ ]:
n_values = [2000000]
benchmark_results = run_benchmark(n_values)

## 2.3) Plotten der Ergebnisse

In [ ]:
avg_times = [result[1] for result in benchmark_results]
std_times = [result[2] for result in benchmark_results]

plt.figure(figsize=(10, 6))
plt.errorbar(n_values, avg_times, yerr=std_times, fmt='-o', ecolor='r', capsize=5, label='Durchschnittliche Rechenzeit mit Standardabweichung')
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Anzahl der Beobachtungen (n)')
plt.ylabel('Rechenzeit (s)')
plt.title('Rechenzeit der QR-Dekompositions-basierten Regression für wachsendes n')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from typing import Dict, List

def format_benchmark_summary(results: pd.DataFrame) -> str:
    """
    Format benchmark results into a readable text summary.
    
    Args:
        results: DataFrame containing benchmark results
    
    Returns:
        Formatted string containing the summary
    """
    summary = []
    summary.append("==============================================================================")
    summary.append("                              Benchmark Results                                ")
    summary.append("==============================================================================")
    
    # Overall statistics
    total_samples = len(results)
    avg_r_squared = results['mean_r_squared'].mean()
    total_successful = results['successful_runs'].sum()
    
    summary.append(f"Total test points:               {total_samples}")
    summary.append(f"Average R-squared:               {avg_r_squared:.4f}")
    summary.append(f"Total successful runs:           {total_successful}")
    summary.append("------------------------------------------------------------------------------")
    summary.append("   Samples    Features     Time (s)    Std Time    R-squared    Std R²")
    summary.append("------------------------------------------------------------------------------")
    
    for _, row in results.iterrows():
        summary.append(
            f"{row['n_samples']:>10,d} "
            f"{row['n_features']:>10d} "
            f"{row['mean_time']:>11.4f} "
            f"{row['std_time']:>11.4f} "
            f"{row['mean_r_squared']:>11.4f} "
            f"{row['std_r_squared']:>10.4f}"
        )
    
    summary.append("==============================================================================")
    return "\n".join(summary)

def plot_benchmark_results(results: pd.DataFrame, plot_type: str = 'all'):
    """
    Plot benchmark results with various visualizations.
    
    Args:
        results: DataFrame containing benchmark results
        plot_type: Type of plot to generate ('time', 'r_squared', or 'all')
    """
    if plot_type in ['time', 'all']:
        # Computation time plot
        plt.figure(figsize=(10, 6))
        plt.errorbar(
            results['n_samples'], 
            results['mean_time'],
            yerr=results['std_time'],
            fmt='o-',
            capsize=5,
            label='Computation Time'
        )
        plt.xscale('log')
        plt.yscale('log')
        plt.xlabel('Sample Size (n)')
        plt.ylabel('Computation Time (s)')
        plt.title('Benchmarking Computation Time by Sample Size')
        plt.grid(True, which="both", ls="-", alpha=0.2)
        plt.legend()
        plt.tight_layout()
        plt.show()
    
    if plot_type in ['r_squared', 'all']:
        # R-squared plot
        plt.figure(figsize=(10, 6))
        plt.errorbar(
            results['n_samples'],
            results['mean_r_squared'],
            yerr=results['std_r_squared'],
            fmt='o-',
            capsize=5,
            color='green',
            label='R-squared'
        )
        plt.xscale('log')
        plt.xlabel('Sample Size (n)')
        plt.ylabel('R-squared')
        plt.title('Model R-squared by Sample Size')
        plt.grid(True, which="both", ls="-", alpha=0.2)
        plt.legend()
        plt.tight_layout()
        plt.show()

def plot_benchmark_comparison(results: List[pd.DataFrame], labels: List[str]):
    """
    Plot benchmark results comparing multiple implementations or configurations.
    
    Args:
        results: List of DataFrames containing benchmark results for different implementations
        labels: List of labels for each implementation
    """
    plt.figure(figsize=(12, 6))
    
    for df, label in zip(results, labels):
        plt.errorbar(
            df['n_samples'],
            df['mean_time'],
            yerr=df['std_time'],
            fmt='o-',
            capsize=5,
            label=label
        )
    
    plt.xscale('log')
    plt.yscale('log')
    plt.xlabel('Sample Size (n)')
    plt.ylabel('Computation Time (s)')
    plt.title('Benchmark Comparison')
    plt.grid(True, which="both", ls="-", alpha=0.2)
    plt.legend()
    plt.tight_layout()
    plt.show()

In [18]:
results_df = run_benchmark_point(n=100000, p=n_features, beta_true=beta_true)

# Print formatted summary
print(format_benchmark_summary(results_df))

# Plot results
plot_benchmark_results(results_df, plot_type='all')


Benchmarking n=100,000 samples...


  Run 1: time=3.22s, R²=0.9994

Summary for n=100,000:
  Mean time: 3.22s ± 0.00s
  Mean R²: 0.9994 ± 0.0000


NameError: name 'format_benchmark_summary' is not defined

Exception in thread "serve RDD 1117" java.net.SocketTimeoutException: Accept timed out
	at java.net.PlainSocketImpl.socketAccept(Native Method)
	at java.net.AbstractPlainSocketImpl.accept(AbstractPlainSocketImpl.java:409)
	at java.net.ServerSocket.implAccept(ServerSocket.java:560)
	at java.net.ServerSocket.accept(ServerSocket.java:528)
	at org.apache.spark.security.SocketAuthServer$$anon$1.run(SocketAuthServer.scala:65)


# 3) Vergleich mit statmodels

## 3.1) statmodels Ausgabe

In [ ]:
X, y = create_data(10000, 7, [-8, -1.6, 4.1, -10, -9.2, 1.3, 1.6, 2.3])
model = sm.OLS(y, X)
results = model.fit()

print(results.summary())

## 3.2) Ausgabe der implementierten Methode

In [ ]:
manual, _ = linear_regression_manual_qr(X,y)

print("\nErgebnisse der manuellen Berechnung:\n")

for key, value in manual[0].items():
    print(f"{key}: {value}")

print("-" * 50)  # 50 Zeichen lange Linie

for key, value in manual[1].items():
    print(f"{key}: {value}")

print("-" * 50)  # 50 Zeichen lange Linie

df = pd.DataFrame(manual[2])
print(df)

print("-" * 50)  # 50 Zeichen lange Linie

for key, value in manual[3].items():
    print(f"{key}: {value}")